In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display

def plot_coil_cylindrical(ax, radius, turns, height, z_offset, color='blue', resolution=100):
    # Angle values: span total angular distance over given number of turns
    theta = np.linspace(0, 2 * np.pi * turns, turns * resolution)
    # Vertical coordinate grows linearly with coil height
    z = np.linspace(0, height, turns * resolution)
    # Project cylindrical coordinates to Cartesian
    x = radius * np.cos(theta)
    y = radius * np.sin(theta)
    # Render helix curve on the 3D axis with offset in z
    ax.plot(x, y, z + z_offset, label='Coil', color=color)

# Widget label styling so descriptors are fully visible
SLIDER_STYLE  = {'description_width': '120px'}
# Slider track sizing for consistent width
SLIDER_LAYOUT = widgets.Layout(width='380px')
# Color picker width to align with sliders
PICKER_LAYOUT = widgets.Layout(width='220px')
# Consistent group width for coil parameter blocks
GROUP_WIDTH   = '540px'

# Distinct preset colors to differentiate coils visually
coil_colors = ['red', 'green', 'purple', 'orange', 'yellow']


In [2]:
# Construct interactive parameter sets for five coils
coil_sliders = []
for i in range(5):
    # Radius of coil in mm
    coil_radius_slider = widgets.FloatSlider(
        value=15, min=1, max=50, step=0.1,
        description='Radius (mm):',
        style=SLIDER_STYLE, layout=SLIDER_LAYOUT, continuous_update=True
    )
    # Number of turns (discrete integer)
    coil_turns_slider = widgets.IntSlider(
        value=5, min=1, max=70, step=1,
        description='Turns:',
        style=SLIDER_STYLE, layout=SLIDER_LAYOUT, continuous_update=True
    )
    # Total vertical height of coil in mm
    coil_height_slider = widgets.FloatSlider(
        value=10.0, min=1.0, max=50.0, step=0.1,
        description='Height (mm):',
        style=SLIDER_STYLE, layout=SLIDER_LAYOUT, continuous_update=True
    )
    # Offset along z-axis so coils can be stacked
    coil_z_offset_slider = widgets.FloatSlider(
        value=0.0, min=0, max=100, step=0.1,
        description='Z Offset (mm):',
        style=SLIDER_STYLE, layout=SLIDER_LAYOUT, continuous_update=True
    )
    # Color picker for visual distinction
    coil_color = widgets.ColorPicker(
        value=coil_colors[i], description='Color:',
        style={'description_width': '60px'}, layout=PICKER_LAYOUT
    )
    # Collect all parameters for this coil into one list
    coil_sliders.append([coil_radius_slider, coil_turns_slider,
                         coil_height_slider, coil_z_offset_slider, coil_color])

# Generate titles "Coil 1" .. "Coil 5" for labeling
coil_titles = [f'Coil {i+1}' for i in range(5)]
# Label widgets to sit above each slider group
coil_title_labels = [widgets.Label(value=title, layout=widgets.Layout(margin='0 0 10px 0'))
                     for title in coil_titles]

# Vertical box: one title plus all sliders for a coil
coil_vboxes = [widgets.VBox([title_label] + sliders, layout=widgets.Layout(width=GROUP_WIDTH))
               for title_label, sliders in zip(coil_title_labels, coil_sliders)]

# Stack groups in a single column for clarity
slider_columns = widgets.GridBox(
    coil_vboxes,
    layout=widgets.Layout(grid_template_columns="repeat(1, 1fr)", width=GROUP_WIDTH)
)

# Reset any pre-existing matplotlib figures to avoid duplicates
plt.close('all')
# Output container that will hold the interactive canvas
fig_out = widgets.Output()
# Create figure + 3D axes, suppressing immediate display
with plt.ioff():
    fig_3d = plt.figure(figsize=(8, 8))
    ax_3d = fig_3d.add_subplot(111, projection='3d')
# Display only the canvas inside the widget output box
with fig_out:
    display(fig_3d.canvas)

# Arrange slider column (left) and figure canvas (right) side by side
display(
    widgets.HBox(
        [slider_columns, fig_out],
        layout=widgets.Layout(align_items='flex-start', gap='16px')
    )
)


In [3]:
def update_plot(change):
    # Default axis extents, expanded only if coils exceed these ranges
    DEFAULT_XY_HALF = 25
    DEFAULT_Z_MAX   = 100
    PAD = 0.10  # multiplicative buffer when enlarging limits

    # Pull current widget values into a clean dictionary list
    coil_data = []
    for sliders in coil_sliders:
        radius, turns, height, z_offset, color = sliders
        coil_data.append({
            'radius': radius.value,
            'turns': turns.value,
            'height': height.value,
            'z_offset': z_offset.value,
            'color': color.value
        })

    # Clear axis and redraw each coil with updated parameters
    ax_3d.clear()
    for data in coil_data:
        plot_coil_cylindrical(ax_3d,
                              data['radius'],
                              data['turns'],
                              data['height'],
                              data['z_offset'],
                              color=data['color'])

    # Calculate bounds from largest radius and highest coil top
    max_radius = max([d['radius'] for d in coil_data] + [0])
    max_z_top  = max([d['z_offset'] + d['height'] for d in coil_data] + [0])

    # X/Y limits: at least default, otherwise expand with buffer
    xy_half = DEFAULT_XY_HALF
    if max_radius > DEFAULT_XY_HALF:
        xy_half = max_radius * (1 + PAD)
    ax_3d.set_xlim([-xy_half, xy_half])
    ax_3d.set_ylim([-xy_half, xy_half])

    # Z limit: same policy as XY
    z_max = DEFAULT_Z_MAX
    if max_z_top > DEFAULT_Z_MAX:
        z_max = max_z_top * (1 + PAD)
    ax_3d.set_zlim([0, z_max])

    # Equal aspect ratio across axes for undistorted 3D visualization
    ax_3d.set_box_aspect([np.ptp(coord) for coord in
                          [ax_3d.get_xlim(), ax_3d.get_ylim(), ax_3d.get_zlim()]])
    # Axis labels and figure title for interpretability
    ax_3d.set_xlabel('X-axis (mm)')
    ax_3d.set_ylabel('Y-axis (mm)')
    ax_3d.set_zlabel('Z-axis (mm)')
    ax_3d.set_title('Optimised Design in 3D')

    # Efficiently refresh canvas without blocking UI thread
    fig_3d.canvas.draw_idle()

# Attach update function to every slider in every coil
for sliders in coil_sliders:
    for slider in sliders:
        slider.observe(update_plot, names='value')

# Perform initial draw so canvas isn’t empty on load
update_plot(None)
